<a href="https://colab.research.google.com/github/Hristo2076/RecSys/blob/main/Rec_Sys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Знакомство с рекомендательными системами**

1. Использовать датасет MovieLens.
2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:
  *   TF-IDF на тегах и жанрах;
  *   средние оценки (+ median, variance и т. д.) пользователя и фильма.


3. Оценить RMSE на тестовой выборке.

In [ ]:
#imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 


from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler




from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2023-06-02 10:39:58--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.124.113, 74.125.124.101, 74.125.124.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.124.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/85c8r9qkkfa8qi29ubrufr4a8gdjoerr/1685702325000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=1c7b9511-ed75-42ec-90b5-0ca0a029461e [following]
--2023-06-02 10:39:59--  https://doc-0g-84-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/85c8r9qkkfa8qi29ubrufr4a8gdjoerr/1685702325000/02611596255248067438/*/1m0rwReR09achL0xTM6QPoN4tykz5bOMx?uuid=1c7b9511-ed75-42ec-90b5-0ca0a029461e
Resolving doc-0g-84-docs.googleusercontent.com (doc-0g-84-docs.googleusercontent.com)... 74.125.126.132, 2607:f8b0:4001:c1d::84
Connecting to doc-0g

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
replace links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: links.csv               
replace movies.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: movies.csv              
replace ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ratings.csv             
replace tags.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: tags.csv                


In [ ]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
movi_tag = movies.merge(tags, on='movieId')
movi_tag.head() 

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932


In [ ]:
def change_string(s):
    return str(s).replace(' ', '').replace('-', '').replace('|','').lower()

tag_strings = []
movi = []
genre = []
from tqdm import tqdm


for movie, group in tqdm(movi_tag.groupby('title')):

  tag_strings.append(' '.join([change_string(s) for s in group.tag.values]))
  genre.append(' '.join([change_string(s) for s in group.genres.values]))
  movi.append(movie)

100%|██████████| 1572/1572 [00:00<00:00, 9675.00it/s] 


In [ ]:
genre[:10]

['comedydramaromance comedydramaromance comedydramaromance comedydramaromance comedydramaromance comedydramaromance comedydramaromance comedydramaromance',
 'dramathriller',
 'thriller thriller',
 'comedyromance',
 'adventurechildrencomedy adventurechildrencomedy',
 'adventureanimationchildren',
 'drama',
 'drama drama drama drama drama drama drama drama drama',
 'adventuredramathriller',
 'comedyfantasyromance']

In [ ]:
tag_strings[:10]

['artistic funny humorous inspiring intelligent quirky romance zooeydeschanel',
 'lawyers',
 'creepy suspense',
 'shakespearesortof',
 'dogs remake',
 'disney',
 'terrorism',
 'court claustrophobic confrontational earnest gooddialogue greatscreenplay gritty motivational thoughtprovoking',
 'stranded',
 'markruffalo']

In [ ]:
tfidf_tag = TfidfVectorizer()
X_train_tfidf_tag = tfidf_tag.fit_transform(tag_strings,genre)
X_train_tfidf_tag

<1572x1472 sparse matrix of type '<class 'numpy.float64'>'
	with 3598 stored elements in Compressed Sparse Row format>

In [ ]:
res = pd.DataFrame(X_train_tfidf_tag.toarray(), columns=tfidf_tag.get_feature_names_out())

res.head()

,06oscarnominatedbestmovieanimation,1900s,1920s,1950s,1960s,1970s,1980s,1990s,2001like,2danimation,70mm,80,aardman,abortion,absorbing,abstract,academyaward,accident,achronological,acleverchefrat,acting,action,actionchoreography,actionpacked,adamsandler,addiction,adingoatemybaby,adolescence,adoption,adorable,adrienbrody,adultery,adulthumor,adventure,afghanistan,africa,agathachristie,aggressive,aging,aids,alanrickman,alcatraz,alcoholism,alfredhitchcock,aliciavikander,aliens,allegorical,aloneintheworld,alpacino,alterego,alternateendings,alternatereality,alternateuniverse,amazing,amazingartwork,amazingcinematography,amazingdialogues,americanidolatry,americanindians,americanpropaganda,amish,amnesia,amtrak,amyadams,ancientrome,andrewlloydweber,android,androids,andygarcia,andykaufman,andysamberg,angelinajolie,anger,animalmovie,animation,anime,anneboleyn,annehathaway,annoying,anthology,anthonyhopkins,antiintellectual,antisemitism,antiwar,apes,apocalypse,aquarium,archaeology,arkofthecovenant,arnoldschwarzenegger,art,arthouse,arthurc,arthurmiller,artificialintelligence,artistic,artsy,asbyatt,assassin,assassination,assassinintraining,assassins,astaireandrogers,asylum,atmospheric,atomicbomb,audienceintelligenceunderestimated,audreytautou,austere,australia,autism,avantgarderomanticcomedy,aviation,awesome,awkward,awkwardromance,babies,backwards,bad,badacting,badass,baddialogue,badhumor,badjokes,badlanguage,badmusic,badplot,badscience,badscript,badstory,badwriting,ballet,ballroomdance,bank,baseball,basedonabook,basedonaplay,basedonatruestory,basedonatvshow,basketball,batman,bears,beatles,beatpoetry,beautiful,beautifulcinematography,beautifullyfilmed,beautifulscenery,beautifulvisuals,bechdeltest,beethoven,benaffleck,benkingsley,benstiller,bestactress,bestcinematography,bestcomedy,besteffectsvisualeffects,bestmusicoriginalscore,bestpicture,bestsupportingactress,bettedavis,bible,biblicalreferences,bigboyswithguns,bigbrothers,bigbudget,bigcorporations,bignameactors,bigtop,bigwave,biking,billmurray,biography,biopic,birds,bittersweet,bizarre,bizzare,blackandwhite,blackcomedy,blackhole,blackhumor,blackhumour,bleak,blind,blindness,blood,bloodsplatters,bloody,bluegrass,boksdrama,bombs,books,borg,boring,boston,bowling,boxing,boxingstory,bradpitt,brainwashing,brilliant,british,britishcomedy,britishgangster,britpop,brittanymurphy,broadway,bromance,bromantic,brooch,brothers,brucewillis,brutal,brutality,bubbagumpshrimp,bugsbunny,buildingafamily,bus,business,busniess,butler,california,cambodia,camels,cameo,camp,canada,cancer,capone,capote,captainamerica,captainkirk,captainnemo,carnival,cartoon,caseyaffleck,casino,casualviolence,cateblanchett,cattledrive,celebrityfetishism,cerebral,cgi,challenging,characterdevelopment,characters,characterstudy,charlesdickens,charlizetheron,charlottebronte,cheating,cheeky,cheesy,chess,chickflick,childabuse,childishnaivity,children,chile,chilly,china,chrisevans,chrisklein,christianbale,christinaricci,christmas,christopherlloyd,christophernolan,christophwaltz,chuckpalahniuk,cia,cinematography,circus,citypolitics,civilwar,claimstobetrue,clarke,class,classic,classicmovie,classicscifi,claustrophobic,claymation,clever,clichecharacters,clock,clousseau,coenbros,coenbrothers,coke,cold,coldwar,coleporter,colinfarrell,college,colorful,colourful,coma,comedy,comicbook,comics,complicated,computer,computeranimation,computers,conan,conartists,confrontational,confusing,confusingending,conmen,conspiracy,conspiracytheory,consumerism,contemplative,controversial,convent,conversation,cool,coolstyle,corruption,costumedrama,couldabeenacontender,court,courtroomdrama,crappysequel,crazy,creative,creativity,creaturefeature,creepy,crime,crimescenescrubbing,crossdressing,crucifixion,crudehumor,cruelcharacters,cryptic,cult,cultclassic,cultfilm,cyberpunk,cyborg,cynical,danaykroyd,dance,dancemarathon,dancing,danielcraig,danielradcliffe,dark,darkcomedy,darkfairytale,darkhero,darkhumor,darthvader,dating,davidbowie,davidfincher,davidthewlis,dayandhudson,dc,dcc

In [ ]:
movi_rat = movies.merge(ratings, on='movieId')

movi_rat.head() 

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
movi_rat['avg_result'] = movi_rat.groupby(['userId'])['rating'].transform('mean')

In [ ]:
movi_rat

,movieId,title,genres,userId,rating,timestamp,avg_result
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,4.366379
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,3.636364
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946,3.230263
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970,3.448148
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483,4.209524
...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,4.0,1537109082,3.705224
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,3.5,1537109545,3.705224
100833,193585,Flint (2017),Drama,184,3.5,1537109805,3.705224
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,3.5,1537110021,3.705224


In [ ]:
X = pd.concat([res,movi_rat.avg_result],axis=1).dropna()
y = movi_rat.rating[:1572] 

In [ ]:
y.shape

(1572,)

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,
                                                 test_size=0.25,
                                                 random_state=24
                                                 )

In [ ]:
pipe = make_pipeline(StandardScaler(), LinearRegression())

pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearregression', LinearRegression())])

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test,y_pred,squared=False)

36656681954006.38